## 1、为什么提示词效果不稳定？
- 1）语义模糊或歧义: 提示词表达不清晰,模型难以理解用户的真实意图
- 2）缺乏上下文支撑: 单一提示鼓励存在,缺少背景信息,导致生成内容偏离预期
- 3）模型对提示词敏感度高: 微小的措辞变化可能导致输出结果大相径庭
- 4）任务复杂度高: 如摘要、推理、多跳问答等任务,仅靠简单提示难以引导模型正确推理

## 2、什么是"上下文融合的Prompt工程"?
- 上下文融合(Contextual Prompt Engineering): 在构造提示时,主动引入相关的背景信息、历史对话、示例、知识片段等,增强大模型对任务的理解和推理能力
- 核心思想: 让大模型"站在更完整的语境中思考",而非仅仅依赖于孤立的指令

## 3、上下文融合的常见策略

### 1) Few-Shot Prompting(少样本提示)
- 少样本提示: 提供若干输入-输出的示例,帮助模型理解任务模式

```md
> 示例：
> 输入：“这部电影太棒了，演员表现非常出色。”
> 输出：“正面评价”
>
> 输入：“服务很差，等了两个小时都没上菜。”
> 输出：“负面评价”
>
> 输入：“天气还可以，但交通有点堵。”
> 输出: ...
```
=》大模型更易推断出要输"中性评价"

### 2) Chain-of-Thought(思维链)
- 引导大模型逐步推理,尤其适用于数学、逻辑类任务

> 示例：
> “小明有5个苹果，吃了2个，又买了3个，现在有多少个？
> 我们一步步思考：
> 1. 起始有5个苹果
> 2. 吃了2个 → 剩下3个
> 3. 又买了3个 → 3 + 3 = 6个
> 所以答案是6。

### 3) 引入外部知识或背景信息
- 将相关文档、数据库片段、用户画像等融入提示词中

> 场景：客服机器人回答产品问题
> 提示中嵌入：“根据《产品手册v2.3》第5章，该设备支持IP67防水标准……”

### 4) 对话历史融合
- 在多轮对话中,将前序交互作为上下文拼接当前提示

> 用户：推荐一部科幻电影
> AI：《星际穿越》
> 用户：有没有类似的？
> → 新提示应包含前一轮问答，避免重复推荐或偏离主题。

### 5) 结构化提示模板
- 使用标准化格式(如JSON、XML或自然语言模板) 组织上下文

```text
【任务】情感分析
【上下文】用户刚完成一次购物，评价如下：
“包装破损，但客服处理很快，补发了新品。”
【示例】类似情况：“物流慢但退款及时” → “中性偏正”
【输出格式】仅输出：正面 / 负面 / 中性
```

## 4、必须要关注的技术细节
- 1）明确任务目标: 先定义清楚大模型你要做什么
- 2）收集相关上下文: 历史对话、用户数据、知识库条目等
- 3）设计提示结构: 分块组织(指令、背景、示例、约束)
- 4）迭代优化: 通过A/B测试对比不同上下文组合的效果
- 5）控制长度: 避免上下文过长导致信息稀释或超出Token限制

## 5、代码实践

In [8]:
import os, dotenv
from openai import OpenAI

# 加载环境变量
dotenv.load_dotenv()
client = OpenAI(
    base_url=os.getenv('BASE_URL'),
    api_key=os.getenv('DEEPSEEK_API_KEY'),
)

def query(user_prompt):
    """
    发送用户提示到 OpenAI API 并返回响应内容

    参数:
        user_prompt (str): 用户输入的提示内容

    返回:
        str: AI 的响应内容
    """
    try:
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "user", "content": user_prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"错误: {str(e)}"


class ContextTemplateFactory:
    """上下文模板工厂类 - 负责创建和管理不同类型的上下文模板"""

    @staticmethod
    def create_customer_feedback_template():
        """创建客户反馈分析模板"""
        return {
            "name": "客户反馈分析",
            "template": """请从客户反馈中提取主要问题，参考以下分类标准：
            1. 物流问题：配送延迟、包装损坏、配送地址错误等
            2. 产品问题：质量缺陷、功能不符、规格错误等
            3. 服务问题：客服响应慢、态度不好、解决方案不满意等
            4. 价格问题：价格变动、优惠券问题、退款问题等

            示例分析：
            原文："快递三天了还没到，客服说在路上，态度还挺好的"
            分析结果：物流延迟（主要问题）+ 客服响应快且态度好（正面反馈）

            现在请分析以下客户反馈：
            {user_input}""",
            "keywords": ["快递", "客服", "配送", "延迟", "态度", "服务", "物流", "反馈", "评价", "投诉", "建议"]
        }

    @staticmethod
    def create_technical_doc_template():
        """创建技术文档总结模板"""
        return {
            "name": "技术文档总结",
            "template": """请按照以下结构总结技术文档：
            1. 核心概念：提取3-5个关键技术概念
            2. 主要功能：列出核心功能特性
            3. 使用场景：说明适用的业务场景
            4. 注意事项：标注重要的限制或注意点

            参考格式：
            - 核心概念：[概念1], [概念2], [概念3]
            - 主要功能：[功能描述]
            - 使用场景：[场景描述]
            - 注意事项：[注意点]

            请总结以下内容：
            {user_input}""",
            "keywords": ["技术", "文档", "API", "框架", "算法", "接口", "认证", "请求", "错误处理"]
        }

    @staticmethod
    def create_default_template():
        """创建默认通用模板"""
        return {
            "name": "通用总结",
            "template": """请对以下内容进行总结：
            {user_input}""",
            "keywords": []
        }

    @classmethod
    def get_all_templates(cls):
        """获取所有可用的模板"""
        return {
            "客户反馈分析": cls.create_customer_feedback_template(),
            "技术文档总结": cls.create_technical_doc_template(),
            "通用总结": cls.create_default_template()
        }


class DynamicContextInjector:
    """
    动态上下文注入器 - 使用模板方法模式定义处理流程

    处理流程：
    1. 初始化模板库
    2. 检测任务类型
    3. 选择合适模板
    4. 注入上下文信息
    5. 执行AI查询
    """

    def __init__(self):
        """初始化：加载所有上下文模板"""
        self.context_templates = self._initialize_templates()

    def _initialize_templates(self):
        """步骤1：初始化模板库（模板方法的具体步骤）"""
        print("正在初始化上下文模板库...")
        templates = ContextTemplateFactory.get_all_templates()
        print(f"成功加载 {len(templates)} 个模板类型")
        return templates

    def _detect_task_type(self, user_input):
        """步骤2：检测任务类型（模板方法的具体步骤）"""
        print(f"正在分析输入内容：{user_input[:30]}...")

        user_input_lower = user_input.lower()

        # 遍历所有模板，寻找关键词匹配
        for template_name, template_config in self.context_templates.items():
            if template_name == "通用总结":  # 跳过默认模板
                continue

            for keyword in template_config["keywords"]:
                if keyword in user_input_lower:
                    print(f"检测到关键词 '{keyword}'，匹配模板：{template_name}")
                    return template_name

        print("未找到特定关键词，使用通用总结模板")
        return "通用总结"

    def _select_template(self, task_type):
        """步骤3：选择合适的模板（模板方法的具体步骤）"""
        print(f"选择模板类型：{task_type}")

        selected_template = self.context_templates.get(task_type)
        if not selected_template:
            print("模板不存在，回退到通用模板")
            selected_template = self.context_templates["通用总结"]

        return selected_template

    def _inject_context(self, user_input, template_config):
        """步骤4：注入上下文信息（模板方法的具体步骤）"""
        print("正在注入上下文信息...")

        template_content = template_config["template"]
        enhanced_prompt = template_content.format(user_input=user_input)

        print(f"增强后提示词长度：{len(enhanced_prompt)} 字符")
        return enhanced_prompt

    def _execute_query(self, enhanced_prompt):
        """步骤5：执行AI查询（模板方法的具体步骤）"""
        print("正在调用AI模型...")
        print("-" * 50)

        result = query(enhanced_prompt)
        return result

    def process_with_context(self, user_input, task_type=None):
        """
        模板方法：定义完整的处理流程
        这是主要的算法骨架，按固定顺序执行各个步骤
        """
        print("开始动态上下文注入处理流程")
        print("=" * 60)

        try:
            # 步骤1：已在初始化时完成

            # 步骤2：检测任务类型
            if task_type is None:
                detected_type = self._detect_task_type(user_input)
            else:
                detected_type = task_type
                print(f"使用指定任务类型：{detected_type}")

            # 步骤3：选择模板
            template_config = self._select_template(detected_type)

            # 步骤4：注入上下文
            enhanced_prompt = self._inject_context(user_input, template_config)

            # 步骤5：执行查询
            result = self._execute_query(enhanced_prompt)

            print("处理流程完成")
            return result

        except Exception as e:
            print(f"处理过程中出现错误：{str(e)}")
            return f"处理失败：{str(e)}"

    # 保持向后兼容的方法名
    def query_with_context(self, user_input, task_type=None):
        """向后兼容的方法名"""
        return self.process_with_context(user_input, task_type)


def demo_comparison():
    """演示基础Prompt vs 上下文融合Prompt的效果对比"""
    injector = DynamicContextInjector()

    # 测试用例
    test_cases = [
        "快递三天了还没到，客服说在路上，态度还挺好的，但是我很着急用这个东西",
        "这个API文档介绍了RESTful接口的使用方法，包括认证、请求格式和错误处理"
    ]

    for i, test_input in enumerate(test_cases, 1):
        print(f"\n=== 测试用例 {i} ===")
        print(f"原始输入: {test_input}")

        # 基础Prompt
        print("\n【未融合Prompt结果】:")
        basic_result = query(f"总结这段话: {test_input}")
        print(basic_result)

        # 上下文融合Prompt
        print("\n【上下文融合Prompt结果】:")
        context_result = injector.query_with_context(test_input)
        print(context_result)

        print("\n" + "=" * 80)


print("\n~~~~ 动态上下文注入演示 ~~~")
# 动态上下文注入演示
demo_comparison()


~~~~ 动态上下文注入演示 ~~~
正在初始化上下文模板库...
成功加载 3 个模板类型

=== 测试用例 1 ===
原始输入: 快递三天了还没到，客服说在路上，态度还挺好的，但是我很着急用这个东西

【未融合Prompt结果】:
好的，这段话的核心意思是：

**您对快递延误感到非常焦急，尽管客服态度良好，但无法解决您的实际问题。**

可以总结为以下几个要点：

1.  **核心问题**：快递物流异常，已延误三天未送达。
2.  **您的感受**：非常着急，因为包裹里的物品急需使用。
3.  **沟通情况**：已联系客服，对方态度友好，但给出的“在路上”这个答复信息模糊，无法缓解您的焦虑。

【上下文融合Prompt结果】:
开始动态上下文注入处理流程
正在分析输入内容：快递三天了还没到，客服说在路上，态度还挺好的，但是我很着急用...
检测到关键词 '快递'，匹配模板：客户反馈分析
选择模板类型：客户反馈分析
正在注入上下文信息...
增强后提示词长度：351 字符
正在调用AI模型...
--------------------------------------------------
处理流程完成
根据客户反馈，分析结果如下：

**主要问题：**
- **物流问题**：快递三天未送达，属于配送延迟

**其他信息：**
- 客服态度良好（正面反馈）
- 客户急需使用该产品（补充说明）

**分析说明：**
虽然客户提到了客服态度良好这一正面反馈，但核心问题仍然是物流配送延迟，这给客户带来了实际困扰，特别是客户明确表示急需使用该产品。客服的良好态度只是服务过程中的一个积极因素，但并未解决客户的核心诉求。


=== 测试用例 2 ===
原始输入: 这个API文档介绍了RESTful接口的使用方法，包括认证、请求格式和错误处理

【未融合Prompt结果】:
好的，这句话可以总结为：

**该API文档说明了如何使用RESTful接口，核心内容包括身份认证、请求格式规范以及错误代码处理机制。**

或者更精炼一点：

**本文档概述了RESTful API的调用方法，重点涵盖认证、请求格式和错误处理。**

**核心要点：**
*   **主题：** RESTful API 使用指南
*   **三大核心内容：**
    1.  **认证：*

## 5、总结

- 动态上下文注入效果对比(示例)

```markdown
| 方法 | 输入提示 | 输出质量 |
|------|---------|----------|
| 基础Prompt | “总结这段话” | 概括不全，遗漏重点 |
| 上下文融合Prompt | “请从客户反馈中提取主要问题，参考以下分类标准：……” + 示例 + 原文 | 准确识别“物流延迟”“客服响应快”等关键点 |
```

- 上下文融合Prompt工程是提升大模型生成质量的关键提示词手段,通过增强语境理解、减少歧义、引导推理路径,提示词的稳定性和输出的可靠性才得以显著提升
- 关键口诀: "不要只给指令,要给场景/示例;不要孤立试探,要贯通上下文"